In [ ]:
#carla_logger.py

# All necessary imports for the logger script
import carla
import time
import sys
import datetime

In [ ]:
# Add CARLA PythonAPI to the system path
# Adjust the path if your CARLA installation is different
try:
    sys.path.append('C:/CARLA_0.9.14/PythonAPI/carla')
except IndexError:
    pass

In [ ]:
# --- Logging function ---
def log_movement(log_file, timestamp, position, direction):
    """
    Logs the car's movement details to a file.
    """
    with open(log_file, 'a') as f:
        log_entry = (
            f"Timestamp: {timestamp}\n"
            f"  - Position:  x={position.x:.2f}, y={position.y:.2f}, z={position.z:.2f}\n"
            f"  - Direction: x={direction.x:.2f}, y={direction.y:.2f}, z={direction.z:.2f}\n"
            f"----------------------------------------\n"
        )
        f.write(log_entry)

In [ ]:
# --- Main logging loop ---
def start_logging(log_file='car_movement_log.txt'):
    try:
        # Connect to the CARLA client
        client = carla.Client('localhost', 2000)
        client.set_timeout(10.0)
        world = client.get_world()

        # Get a reference to the vehicle.
        # This assumes the main script has already spawned a vehicle.
        # We look for the first vehicle actor in the world.
        vehicle_list = world.get_actors().filter('vehicle.*')
        
        if not vehicle_list:
            print("No vehicle found in the simulation. Please run the main controller script first.")
            return

        vehicle = vehicle_list[0]
        print(f"Logger connected to vehicle: {vehicle.type_id}")

        # --- Initial logging ---
        initial_pos = vehicle.get_transform().location
        initial_dir = vehicle.get_transform().get_forward_vector()
        with open(log_file, 'w') as f:
            f.write(f"--- Simulation Log Start ---\n")
            f.write(f"Initial Position: x={initial_pos.x:.2f}, y={initial_pos.y:.2f}, z={initial_pos.z:.2f}\n")
            f.write(f"Initial Direction: x={initial_dir.x:.2f}, y={initial_dir.y:.2f}, z={initial_dir.z:.2f}\n")
            f.write(f"----------------------------------------\n")
        
        print("Logging started. Press Ctrl+C to stop.")

        # --- Logging loop ---
        while True:
            # Get current vehicle transform
            vehicle_transform = vehicle.get_transform()
            current_pos = vehicle_transform.location
            current_dir = vehicle_transform.get_forward_vector()
            timestamp = datetime.datetime.now().isoformat()

            # Log the data
            log_movement(log_file, timestamp, current_pos, current_dir)

            # Wait for a short duration before the next log entry
            time.sleep(0.1) # Log every 100ms
            
    except KeyboardInterrupt:
        print("\nLogging stopped by user.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        print("Logger script finished.")

In [ ]:
if __name__ == '__main__':
    start_logging()